In [265]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle as pk

In [62]:
movie = pd.read_csv('MovieReco2.csv')

In [63]:
a= pd.read_csv('movieMD.csv')
b = pd.read_csv('MDmovie2.csv')

C:\Users\prana\AppData\Local\Temp\ipykernel_4692\2172584221.py:2: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256

In [64]:
movie.head(1)

,Unnamed: 0,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


Now, there will be only 3 columns: id, title, tags. The tags column will be created by combining rest of the columns. In genres: add all the genres, in keywords: Add everything, in cast: Add top 3 actors. At the end we will have a big paragraph.

In [65]:
movie = movie[['genres', 'id',  'keywords' , 'title', 'overview', 'popularity']]

print(movie.head(1))

                                              genres     id  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...  19995   

                                            keywords   title  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...  Avatar   

                                            overview  popularity  
0  In the 22nd century, a paraplegic Marine is di...  150.437577  


In [66]:
movie.isnull().sum()

genres        0
id            0
keywords      0
title         0
overview      3
popularity    0
dtype: int64

In [67]:
movie.dropna(inplace=True)

In [68]:
movie.isna().sum()

genres        0
id            0
keywords      0
title         0
overview      0
popularity    0
dtype: int64

In [69]:
movie.duplicated().sum()

6

In [70]:
def convert(w):
    q = []
    for i in literal_eval(w):
        q.append(i['name'])

    return q    


In [71]:
movie['genres'] = movie['genres'].apply(convert)

In [72]:
movie.head(1)

,genres,id,keywords,title,overview,popularity
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577


In [73]:
movie['keywords'] = movie['keywords'].apply(convert)

In [74]:
movie.head(1)

,genres,id,keywords,title,overview,popularity
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577


In [75]:
def concast(w):
    q = []
    a = 0
    for i in literal_eval(w):
       
        q.append(i['name'])
        a += 1
        if a == 3:
            break;

    return q

In [76]:
movie.head(183)

,genres,id,keywords,title,overview,popularity
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,A cryptic message from Bond’s past sends him o...,107.376788
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"John Carter is a war-weary, former military ca...",43.926995
...,...,...,...,...,...,...
178,"[Animation, Comedy, Family, Western, Adventure]",44896,"[sheriff, nevada, pet, rango, chameleon, las v...",Rango,"When Rango, a lost family pet, accidentally wi...",29.913530
179,"[Family, Animation, Adventure, Comedy]",270946,"[penguin, madagascar, 3d]",Penguins of Madagascar,"Skipper, Kowalski, Rico and Private join force...",84.366984
180,"[Action, Drama, Mystery, Thriller]",2503,"[paris, corruption, madrid, assassin, based on...",The Bourne Ultimatum,Bourne is brought out of hiding once again by ...,45.381501
181,"[Adventure, Animation, Family, Comedy]",9502,"[china, martial arts, kung fu, mentor, snake, ...",Kung Fu Panda,"When the Valley of Peace is threatened, lazy P...",84.689648


In [77]:
movie.size

28824

In [78]:
movie.head(1)

,genres,id,keywords,title,overview,popularity
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577


In [79]:
movie['overview'] = movie['overview'].apply(lambda x: x.split())

In [80]:
movie.head(1)

,genres,id,keywords,title,overview,popularity
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577


In [81]:
#Removing spaces between words. eg science fiction ----> sciencefiction. Because model gives tages for each word 
#and a word can have different association which can change its meaning completely. 


movie['genres'] = movie['genres'].apply(lambda x: [i.replace(" ","") for i in x])

In [82]:
movie['keywords'] = movie['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

In [86]:
movie['overview'] = movie['overview'].apply(lambda x: [i.replace(" ", '') for i in x])

In [87]:
movie.head(1)

,genres,id,keywords,title,overview,popularity
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577


In [88]:
movie['tages'] = movie['genres'] + movie['overview'] + movie['keywords'] 

In [89]:
movie.head(1)

,genres,id,keywords,title,overview,popularity,tages
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[Action, Adventure, Fantasy, ScienceFiction, I..."


In [90]:
movie_ds = movie[['id', 'title', 'tages']]

In [91]:
movie_ds.head(1)

,id,title,tages
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, I..."


In [92]:
#converting list into string
movie_ds['tages'] = movie_ds['tages'].apply(lambda x: " ".join(x))

C:\Users\prana\AppData\Local\Temp\ipykernel_4692\2777028567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_ds['tages'] = movie_ds['tages'].apply(lambda x: " ".join(x))


In [93]:
movie_ds.head(1)

,id,title,tages
0,19995,Avatar,Action Adventure Fantasy ScienceFiction In the...


In [95]:
movie_ds['tages'][1400]

'Crime Drama Thriller An FBI undercover agent infilitrates the mob and finds himself identifying more with the mafia life at the expense of his regular one. undercover colombia mafia mobster dirtycop informant stealingmoney marriagecounselor shavingcream playingcards'

In [102]:
#converting dataset into lowercase
movie_ds['tages'] = movie_ds['tages'].apply(lambda x: x.lower())

C:\Users\prana\AppData\Local\Temp\ipykernel_4692\2347693450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_ds['tages'] = movie_ds['tages'].apply(lambda x: x.lower())


In [104]:
movie_ds['tages'][1400]

'crime drama thriller an fbi undercover agent infilitrates the mob and finds himself identifying more with the mafia life at the expense of his regular one. undercover colombia mafia mobster dirtycop informant stealingmoney marriagecounselor shavingcream playingcards'

**Vectorizaiton**

In [107]:
ve = CountVectorizer(max_features= 5000, stop_words='english')

In [114]:
vectors = ve.fit_transform(movie_ds['tages']).toarray()

In [115]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [118]:
ve.get_feature_names_out()

array(['000', '007', '10', ..., 'zombies', 'zone', 'zoo'], dtype=object)

**Steamming** Removing words with same meaning but different forms. Eg. Actors, Actor

In [121]:
ps = PorterStemmer()

In [126]:
def stem(w):
    t = []

    for i in w.split():
        t.append(ps.stem(i))
      
    return " ".join(t)  # converting List into string

In [128]:
movie_ds['tages'] = movie_ds['tages'].apply(stem)

C:\Users\prana\AppData\Local\Temp\ipykernel_4692\288693025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_ds['tages'] = movie_ds['tages'].apply(stem)


**Cosine Distance**: As in Euclidian diatance, we calculate the distance between the tips of the vector, in Cosine distance we calculate the angle between the vector. Lesser the angle, similar the vector. The Euclidean distance method is not reliable while dealing with huge data set. Hence we use Cosine distance.

In [131]:
cosine_similarity(vectors)

array([[1.        , 0.09583148, 0.06277648, ..., 0.02465568, 0.0270666 ,
        0.        ],
       [0.09583148, 1.        , 0.07018624, ..., 0.02756589, 0.        ,
        0.        ],
       [0.06277648, 0.07018624, 1.        , ..., 0.02708645, 0.        ,
        0.        ],
       ...,
       [0.02465568, 0.02756589, 0.02708645, ..., 1.        , 0.07007128,
        0.04732485],
       [0.0270666 , 0.        , 0.        , ..., 0.07007128, 1.        ,
        0.05195243],
       [0.        , 0.        , 0.        , ..., 0.04732485, 0.05195243,
        1.        ]])

In [132]:
cosine_similarity(vectors).shape

(4804, 4804)

In [133]:
simi = cosine_similarity(vectors)

In [225]:
sorted(list(enumerate(simi[1])),reverse=True, key= lambda x:x[1])

[(1, 0.9999999999999997),
 (12, 0.3471825374147068),
 (17, 0.2683725200608466),
 (187, 0.2268713032432575),
 (3571, 0.22677868380553634),
 (216, 0.21821789023599236),
 (199, 0.21170244960998522),
 (75, 0.20365326999063918),
 (848, 0.20185258680731544),
 (981, 0.19504737440137349),
 (349, 0.1889822365046136),
 (987, 0.1889822365046136),
 (1096, 0.1889822365046136),
 (2134, 0.18708286933869706),
 (4008, 0.18333969940564224),
 (818, 0.181848241863327),
 (310, 0.18028869304837586),
 (1334, 0.17857142857142852),
 (61, 0.1781741612749496),
 (109, 0.1781741612749496),
 (340, 0.1770844008302866),
 (3054, 0.17342199390482396),
 (104, 0.16971105832553265),
 (1194, 0.1690308509457033),
 (190, 0.1653594569415369),
 (1727, 0.1649572197684645),
 (24, 0.16448792373994225),
 (129, 0.1636634176769943),
 (2595, 0.16366341767699427),
 (98, 0.16205093088804107),
 (276, 0.16116459280507603),
 (781, 0.15762208124782012),
 (2660, 0.15762208124782012),
 (1664, 0.15724272550828775),
 (138, 0.1553424415003),
 (

In [260]:
def recommend(w):
    mi = movie_ds[movie_ds['title'] == w].index[0]
    sr = sorted(enumerate(simi[mi]),reverse=True,key= lambda x:x[1])[1:6]
    
    g = []
    for i in sr:
        g.append(i[0])

    for i in g:
        print(movie_ds['title'][i])    

In [245]:
arr = np.array([34, 65, 22, 75, 78, 10, 32, 54])

for i in range(len(arr)):
    for j in range(len(arr)):
        t = 0
        if arr[i] < arr[j]:
            t = arr[j]
            arr[j] = arr[i]
            arr[i] = t

print(arr)    


p = sorted(list(enumerate(arr)),key = lambda x:x[1])
print(p)

[10 22 32 34 54 65 75 78]
[(0, 10), (1, 22), (2, 32), (3, 34), (4, 54), (5, 65), (6, 75), (7, 78)]


In [184]:
simi[3]

array([0.040996  , 0.02291746, 0.04503773, ..., 0.05306619, 0.05825517,
       0.05901671])

In [264]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
Batman & Robin
The Dark Knight Rises


In [232]:
movie_ds['title'][10:100]

10                              Superman Returns
11                             Quantum of Solace
12    Pirates of the Caribbean: Dead Man's Chest
13                               The Lone Ranger
14                                  Man of Steel
                         ...                    
95                                  Interstellar
96                                     Inception
97                                 Shin Godzilla
98             The Hobbit: An Unexpected Journey
99                      The Fast and the Furious
Name: title, Length: 90, dtype: object

In [258]:
movie_ds['title'][101]

'X-Men: First Class'

In [268]:
pk.dump(movie_ds.to_dict(),open('chitrapat_dakhva.pkl', 'wb'))

In [269]:
pk.dump(simi,open('simimovie.pkl','wb'))